In [1]:
# drazin.py
"""Volume 1: The Drazin Inverse.
<John Wilson>
<7/5/2017>
"""

import numpy as np
from scipy import linalg as la


# Helper function for problems 1 and 2.
def index(A, tol=1e-5):
    """Compute the index of the matrix A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.

    Returns:
        k (int): The index of A.
    """

    # test for non-singularity
    if not np.allclose(la.det(A),0):
        return 0

    n = len(A)
    k = 1
    Ak = A.copy()
    while k <= n:
        r1 = np.linalg.matrix_rank(Ak)
        r2 = np.linalg.matrix_rank(np.dot(A,Ak))
        if r1 == r2:
            return k
        Ak = np.dot(A,Ak)
        k += 1

    return k

In [2]:
# Problem 1
def is_drazin(A, Ad, k):
    """Verify that a matrix Ad is the Drazin inverse of A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.
        Ad ((n,n) ndarray): A candidate for the Drazin inverse of A.
        k (int): The index of A.

    Returns:
        bool: True of Ad is the Drazin inverse of A, False otherwise.
    """
    power=np.linalg.matrix_power
    if not np.allclose(A@Ad,Ad@A):
        return False
    if not np.allclose(power(A,k+1)@Ad,power(A,k)):
        return False
    if not np.allclose(Ad@A@Ad,Ad):
        return False
    return True

In [3]:
# Problem 2
def drazin_inverse(A, tol=1e-4):
    """Compute the Drazin inverse of A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.

    Returns:
        Ad ((n,n) ndarray): The Drazin inverse of A.
    """
    n,n=A.shape
    f = lambda x: abs(x)>tol
    Q1,S,k1 = la.schur(A,sort=f)
    f = lambda x: abs(x)<=tol
    Q2,T,k2 = la.schur(A,sort=f)
    U=np.concatenate((S[:,:k1],T[:,:n-k1]),axis=1)
    U_inv=la.inv(U)
    V=U_inv@A@U
    Z=np.zeros((n,n),dtype=np.float)
    if k1!=0:
        M_inv=la.inv(V[:k1,:k1])
        Z[:k1,:k1]=M_inv
    return U@Z@U_inv

We now test functions for problems 1 and 2, using examples from the lab

In [4]:
def tests(A):
    Ad=drazin_inverse(A)
    print(Ad)
    k=index(A)
    return is_drazin(A,Ad,k)

In [5]:
A=np.array([[1.,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
tests(A)

[[  1.  -3.   9.  81.]
 [  0.   1.  -3. -18.]
 [  0.   0.   1.   3.]
 [  0.   0.   0.   0.]]


True

In [6]:
B = np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
tests(B)

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


True

In [7]:
# Problem 3

#First a helper function I wrote for another lab
def laplacian(A):
    '''
    Compute the Laplacian matrix of the adjacency matrix A.
    Inputs:
        A (array): adjacency matrix for undirected weighted graph,
             shape (n,n)
    Returns:
        L (array): Laplacian matrix of A

    '''
    diags = np.sum(A,axis=1)
    D=np.diag(diags)
    return D-A

def effective_res(A):
    """Compute the effective resistance for each node in a graph.

    Parameters:
        A ((n,n) ndarray): The adjacency matrix of an undirected graph.

    Returns:
        ER ((n,n) ndarray): A matrix of which the ijth entry is the effective
        resistance from node i to node j.
    """
    n = A.shape[0]
    L = laplacian(A)
    R = np.zeros_like(A,dtype=np.float)
    for i in range(A.shape[0]):
        Ltemp = np.copy(L)
        evec = np.zeros(n)
        evec[i] = 1
        Ltemp[i] = evec
        Ld = drazin_inverse(Ltemp)
        R[:,i] = np.diag(Ld)
        R[i,i]=0
    return R

The following are some test cases for the effective resistance function

In [8]:
A = np.array([[0,0,1,0],[0,0,0,1],[1,0,0,1],[0,1,1,0]])
effective_res(A)

array([[ 0.,  3.,  1.,  2.],
       [ 3.,  0.,  2.,  1.],
       [ 1.,  2.,  0.,  1.],
       [ 2.,  1.,  1.,  0.]])

In [9]:
B=np.array([[0,4],[4,0]])
effective_res(B)

array([[ 0.  ,  0.25],
       [ 0.25,  0.  ]])

In [10]:
C = np.array([[0,1,1],[1,0,1],[1,1,0]])
effective_res(C)

array([[ 0.        ,  0.66666667,  0.66666667],
       [ 0.66666667,  0.        ,  0.66666667],
       [ 0.66666667,  0.66666667,  0.        ]])

In [11]:
# Problem 4
class LinkPredictor:
    """Predict links between nodes of a network."""

    def __init__(self, filename='social_network.csv'):
        """Create the effective resistance matrix by constructing
        an adjacency matrix.
        
        Parameters:
            filename (str): The name of a file containing graph data.
        """
        self.fromp=[]
        self.top=[]
        with open(filename, 'r') as infile:
            lines=infile.readlines()
        for l in lines:
            line = l.split(',')
            self.fromp.append(line[0])
            self.top.append(line[1].strip())
        keys=set()
        for p in self.fromp:
            keys.add(p)
        for p in self.top:
            keys.add(p)
        self.key=dict.fromkeys(keys)
        for i,k in enumerate(self.key):
            self.key[k]=i
        A=np.zeros((len(keys),len(keys)))
        for i,l in enumerate(self.fromp):
            A[self.key[l],self.key[self.top[i]]]=1
            A[self.key[self.top[i]],self.key[l]]=1
        self.A=A
        self.R=effective_res(self.A)

In [12]:
f=LinkPredictor()